In [135]:
import numpy as np
import pandas as pd

datapath = './data/Hostel2.csv'#iso-8859-1 에서 utf-8로 인코딩
data1 = pd.read_csv(datapath, sep=',', encoding='utf-8')

# data1.shape

In [136]:
# data1.head()

In [137]:
# data1.isnull().sum()
# #결측치 확인

In [138]:
filtered_data = data1.dropna()#결측값있는 행 제거
del(filtered_data['Unnamed: 0'])
del(filtered_data['hostel.name'])#필요없는 열 제거
filtered_data['Distance']=filtered_data['Distance'].apply(lambda x: x.replace('km from city centre',''))
# filtered_data.head()

/home/u1019/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [139]:
# filtered_data.shape
# filtered_data.info()#결측치 제거 후 데이터 확인

In [140]:
from sklearn import preprocessing 

le_city = preprocessing.LabelEncoder()
le_city.fit(filtered_data['City'])
le_rating = preprocessing.LabelEncoder()
le_rating.fit(filtered_data['rating.band'])


# print(le_city.classes_)
# print(le_city.transform(['Fukuoka-City', 'Hiroshima', 'Kyoto', 'Osaka', 'Tokyo']))
# print(le_city.inverse_transform([0, 1, 2, 3, 4]))
# print(le_rating.classes_)
# print(le_rating.transform(['Fabulous', 'Good', 'Rating', 'Superb', 'Very Good']))
# print(le_rating.inverse_transform([0, 1, 2, 3, 4]))

LabelEncoder()

In [141]:
# filtered_data['price.from'].value_counts()

In [142]:
import numpy as np
a = filtered_data.copy()
a = np.where((a["price.from"].value_counts()==1),1,a["price.from"].value_counts())

In [143]:
# i_class0 = np.where(a!=0)[0]
# i_class1 = np.where(a==1)[0]

# n_class0 = len(i_class0)
# n_class1 = len(i_class1)

# i_class1_upsampled = np.random.choice(i_class1,size=n_class0,replace=True)
# np.concatenate((a[i_class1_upsampled], a[i_class0]))

In [144]:
dicted_data = filtered_data.copy()
dicted_data['City']=le_city.transform(filtered_data['City'])
dicted_data['rating.band']=le_rating.transform(filtered_data['rating.band'])

# dicted_data.head(10)

In [145]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge

features = ['City','rating.band','Distance','summary.score','atmosphere','cleanliness','facilities','location.y','security','staff','valueformoney']

train_d, test_d = train_test_split(dicted_data, test_size=0.5)

train_y = train_d['price.from']
train_x = train_d[features]

test_y = test_d['price.from']
test_x = test_d[features]

models = [LinearRegression(), Ridge()]
for m in models:
    m.fit(train_x, train_y)
#     print('model {}= {}'.format(m, m.score(test_x, test_y)))

In [146]:
from sklearn.model_selection  import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler



features = ['City','rating.band','Distance','summary.score','atmosphere','cleanliness','facilities','location.y','security','staff','valueformoney']

scaler = MinMaxScaler(features)

kf = KFold(n_splits=10, shuffle=True)
print(type(kf))
fold_idx = 1
mae = []

for train_idx, test_idx in kf.split(dicted_data):
    print('[Fold {}] train size = {}, test size = {}'.format(fold_idx, len(train_idx),len(test_idx)))
    train_d, test_d = dicted_data.iloc[train_idx], dicted_data.iloc[test_idx]
    
    train_y = train_d['price.from']
    train_x= train_d[features]
    
    test_y = test_d['price.from']
    test_x = test_d[features]
    
    model = LinearRegression()
    model.fit(train_x, train_y)
    
    mean_mae = model.score(test_x, test_y)
    print('MAE ={}'.format(mean_mae))
    fold_idx += 1 
    mae.append(mean_mae)
    
print(np.average(mae))

# Data imbalance 체크를 통해 빈도수가 적은 label 값을 업샘플링을 통해 
# 데이터 불균형을 해결할 수 있지만 구현하지 못했다.

# 데이터상 label을 예측하는데 필요하지 않은 위도, 경도 feature 값으로 포함시켰을때 
# 성능이 낮아져 feature에 포함시키지 않았고, 카테고리 변수들의 feature를 정규화 하였지만 정규화 하기 전과 성능이 비슷했다.
# 왜냐하면 feature의 범위가 비슷하기 때문이다. 

# 2가지모델(Linear regression, Ridge regression)을 돌려본 결과
# Linear과 Ridge는 성능이 비슷하게 나와 오버피팅이 없는것으로 판단하여 일반적인 Linear regression 모델을 선택하였다. 

<class 'sklearn.model_selection._split.KFold'>
[Fold 1] train size = 268, test size = 30
MAE =-0.12924865165793054
[Fold 2] train size = 268, test size = 30
MAE =0.04692271816142989
[Fold 3] train size = 268, test size = 30
MAE =-0.3074036301455336
[Fold 4] train size = 268, test size = 30
MAE =-0.04322653472621529
[Fold 5] train size = 268, test size = 30
MAE =0.09045404814748215
[Fold 6] train size = 268, test size = 30
MAE =0.11159740105424709
[Fold 7] train size = 268, test size = 30
MAE =-0.18027325461279942
[Fold 8] train size = 268, test size = 30
MAE =-0.021689034525122164
[Fold 9] train size = 269, test size = 29
MAE =0.01929578833215162
[Fold 10] train size = 269, test size = 29
MAE =0.14220306891160406
-0.02713680810606862
